In [2]:
import numpy as np
from scipy.stats import norm

def black_scholes(option_type, S, K, T, r, sigma):
    """
    Calculate the Black-Scholes option price.

    Parameters:
    option_type (str): 'call' for call option, 'put' for put option
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)

    Returns:
    float: option price
    """
    # Calculate d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Calculate call and put option prices
    if option_type == 'call':
        option_price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == 'put':
        option_price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return option_price

# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (1 year)
r = 0.05 # Risk-free interest rate (5%)
sigma = 0.2 # Volatility (20%)

call_price = black_scholes('call', S, K, T, r, sigma)
put_price = black_scholes('put', S, K, T, r, sigma)

print(f"Call Option Price: {call_price:.2f}")
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 10.45
Put Option Price: 5.57


## Consistency test - Gamma Posivity
Performs the test to make sure the vanilla option priver gives positive Gamma under secific contractual conditoins. 
This is done by bumping prices up and down by 1% and checking (priceUp+priceDown-2basePrice) staying positive.


In [3]:
def test_gamma_positivity(option_type, S, K, T, r, sigma):
    """
    Test the gamma positivity of the Black-Scholes model.

    Parameters:
    option_type (str): 'call' for call option, 'put' for put option
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)

    Returns:
    bool: True if gamma positivity holds, False otherwise
    """
    # Calculate the base price
    base_price = black_scholes(option_type, S, K, T, r, sigma)

    # Bump prices up and down by 1%
    bump = S * 0.01
    price_up = black_scholes(option_type, S + bump, K, T, r, sigma)
    price_down = black_scholes(option_type, S - bump, K, T, r, sigma)

    # Check gamma positivity condition
    gamma_condition = price_up + price_down - 2 * base_price
    return gamma_condition > 0

# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (1 year)
r = 0.05 # Risk-free interest rate (5%)
sigma = 0.2 # Volatility (20%)

is_gamma_positive = test_gamma_positivity('call', S, K, T, r, sigma)
print(f"Gamma Positivity Test for Call Option: {'Positive' if is_gamma_positive else 'Negative'}")

is_gamma_positive_put = test_gamma_positivity('put', S, K, T, r, sigma)
print(f"Gamma Positivity Test for Put Option: {'Positive' if is_gamma_positive_put else 'Negative'}")

Gamma Positivity Test for Call Option: Positive
Gamma Positivity Test for Put Option: Positive


## PDE Convergence test
PDE convergence is tested by increasing the grid points in timpe space and stock price space from 100 to 500

In [4]:
import numpy as np

def black_scholes_pde(S_max, K, T, r, sigma, S_points, T_points, option_type):
    """
    Solve the Black-Scholes PDE using finite difference method.

    Parameters:
    S_max (float): maximum stock price
    K (float): option strike price
    T (float): time to expiration
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)
    S_points (int): number of stock price grid points
    T_points (int): number of time grid points
    option_type (str): 'call' for call option, 'put' for put option

    Returns:
    float: option price
    """
    # Create the stock price and time grids
    S = np.linspace(0, S_max, S_points)
    dt = T / T_points
    dS = S_max / S_points

    # Initialize the option price matrix
    V = np.zeros((S_points, T_points + 1))

    # Boundary conditions
    if option_type == 'call':
        V[:, -1] = np.maximum(0, S - K)  # Payoff at maturity for call
    elif option_type == 'put':
        V[:, -1] = np.maximum(0, K - S)  # Payoff at maturity for put

    # Finite difference coefficients
    alpha = 0.5 * dt * (sigma**2 * (np.arange(S_points) ** 2) / (dS ** 2) - r / dS)
    beta = -dt * (sigma**2 * (np.arange(S_points) ** 2) / (dS ** 2) + r)
    gamma = 0.5 * dt * (sigma**2 * (np.arange(S_points) ** 2) / (dS ** 2) + r / dS)

    # Time-stepping backward
    for j in range(T_points - 1, -1, -1):
        for i in range(1, S_points - 1):
            V[i, j] = alpha[i] * V[i - 1, j + 1] + (1 - beta[i]) * V[i, j + 1] + gamma[i] * V[i + 1, j + 1]

    return V[int(S_points / 2), 0]  # Return the option price at S = K

def pde_convergence_test(S_max, K, T, r, sigma, option_type):
    """
    Test the convergence of the PDE solution by varying grid points.

    Parameters:
    S_max (float): maximum stock price
    K (float): option strike price
    T (float): time to expiration
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)
    option_type (str): 'call' for call option, 'put' for put option
    """
    prices = []
    grid_points = range(100, 501, 100)  # From 100 to 500

    for S_points in grid_points:
        T_points = S_points  # Set time points equal to stock points for simplicity
        price = black_scholes_pde(S_max, K, T, r, sigma, S_points, T_points, option_type)
        prices.append(price)

    # Check convergence
    for i in range(1, len(prices)):
        if abs(prices[i] - prices[i - 1]) > 1e-5:  # Convergence threshold
            print(f"Prices do not converge at grid points {grid_points[i-1]} and {grid_points[i]}")
            return

    print("Prices converge across the grid points.")

# Example usage
S_max = 200  # Maximum stock price
K = 100      # Strike price
T = 1        # Time to expiration (1 year)
r = 0.05     # Risk-free interest rate (5%)
sigma = 0.2  # Volatility (20%)

pde_convergence_test(S_max, K, T, r, sigma, 'call')

C:\Users\BB211CT\AppData\Local\Temp\ipykernel_33160\2878715046.py:42: RuntimeWarning: overflow encountered in scalar add
  V[i, j] = alpha[i] * V[i - 1, j + 1] + (1 - beta[i]) * V[i, j + 1] + gamma[i] * V[i + 1, j + 1]
C:\Users\BB211CT\AppData\Local\Temp\ipykernel_33160\2878715046.py:42: RuntimeWarning: overflow encountered in scalar multiply
  V[i, j] = alpha[i] * V[i - 1, j + 1] + (1 - beta[i]) * V[i, j + 1] + gamma[i] * V[i + 1, j + 1]


Prices do not converge at grid points 100 and 200


## Sensitivity test 
Spot smoothness is tested by sliding the spot from -2.5% to 2.5%, with steps of 0.5%. Several quantities are tested, inclduing price, Delta, Gamma, Vega, Rho, RhoDiv and Theta.

In [5]:
def calculate_greeks(option_type, S, K, T, r, sigma):
    """
    Calculate the option price and Greeks using the Black-Scholes model.

    Parameters:
    option_type (str): 'call' for call option, 'put' for put option
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)

    Returns:
    dict: option price and Greeks
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
        delta = norm.cdf(d1)
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == 'put':
        price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))
        delta = norm.cdf(d1) - 1
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2))
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'vega': vega,
        'rho': rho,
        'theta': theta
    }

def sensitivity_test(option_type, S, K, T, r, sigma):
    """
    Perform sensitivity test by sliding the spot price.

    Parameters:
    option_type (str): 'call' for call option, 'put' for put option
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)
    """
    results = []
    spot_changes = np.arange(-0.025, 0.026, 0.005)  # From -2.5% to 2.5% in steps of 0.5%

    for change in spot_changes:
        new_S = S * (1 + change)
        greeks = calculate_greeks(option_type, new_S, K, T, r, sigma)
        results.append({
            'spot_change': change,
            'price': greeks['price'],
            'delta': greeks['delta'],
            'gamma': greeks['gamma'],
            'vega': greeks['vega'],
            'rho': greeks['rho'],
            'theta': greeks['theta']
        })

    return results

# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (1 year)
r = 0.05 # Risk-free interest rate (5%)
sigma = 0.2 # Volatility (20%)

sensitivity_results = sensitivity_test('call', S, K, T, r, sigma)

# Print results
for result in sensitivity_results:
    print(f"Spot Change: {result['spot_change']:.2%}, Price: {result['price']:.2f}, "
          f"Delta: {result['delta']:.4f}, Gamma: {result['gamma']:.4f}, "
          f"Vega: {result['vega']:.4f}, Rho: {result['rho']:.4f}, Theta: {result['theta']:.4f}")

Spot Change: -2.50%, Price: 8.92, Delta: 0.5884, Gamma: 0.0200, Vega: 37.9382, Rho: 48.4498, Theta: -6.2163
Spot Change: -2.00%, Price: 9.22, Delta: 0.5983, Gamma: 0.0197, Vega: 37.9031, Rho: 49.4197, Theta: -6.2613
Spot Change: -1.50%, Price: 9.52, Delta: 0.6081, Gamma: 0.0195, Vega: 37.8436, Rho: 50.3834, Theta: -6.3035
Spot Change: -1.00%, Price: 9.82, Delta: 0.6178, Gamma: 0.0193, Vega: 37.7602, Rho: 51.3405, Theta: -6.3430
Spot Change: -0.50%, Price: 10.13, Delta: 0.6274, Gamma: 0.0190, Vega: 37.6535, Rho: 52.2904, Theta: -6.3799
Spot Change: 0.00%, Price: 10.45, Delta: 0.6368, Gamma: 0.0188, Vega: 37.5240, Rho: 53.2325, Theta: -6.4140
Spot Change: 0.50%, Price: 10.77, Delta: 0.6461, Gamma: 0.0185, Vega: 37.3723, Rho: 54.1664, Theta: -6.4456
Spot Change: 1.00%, Price: 11.10, Delta: 0.6553, Gamma: 0.0182, Vega: 37.1990, Rho: 55.0917, Theta: -6.4745
Spot Change: 1.50%, Price: 11.43, Delta: 0.6644, Gamma: 0.0180, Vega: 37.0047, Rho: 56.0078, Theta: -6.5009
Spot Change: 2.00%, Price: 

## PnL explain test
Greek-based PNL and revaluation based PNL for Delta/Gamma are compared.

In [6]:
def pnl_explain_test(option_type, S, K, T, r, sigma, price_change):
    """
    Perform PnL explain test comparing Greek-based PnL and revaluation-based PnL.

    Parameters:
    option_type (str): 'call' for call option, 'put' for put option
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)
    price_change (float): change in stock price for the test
    """
    # Calculate initial option price and Greeks
    initial_greeks = calculate_greeks(option_type, S, K, T, r, sigma)
    initial_price = initial_greeks['price']
    delta = initial_greeks['delta']
    gamma = initial_greeks['gamma']

    # Calculate Greek-based PnL
    greek_based_pnl = delta * price_change + 0.5 * gamma * (price_change ** 2)

    # Recalculate the option price after the price change
    new_S = S + price_change
    new_price = calculate_greeks(option_type, new_S, K, T, r, sigma)['price']

    # Calculate revaluation-based PnL
    revaluation_based_pnl = new_price - initial_price

    # Print results
    print(f"Initial Price: {initial_price:.2f}")
    print(f"New Price: {new_price:.2f}")
    print(f"Greek-based PnL: {greek_based_pnl:.2f}")
    print(f"Revaluation-based PnL: {revaluation_based_pnl:.2f}")

# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (1 year)
r = 0.05 # Risk-free interest rate (5%)
sigma = 0.2 # Volatility (20%)
price_change = 1.0  # Change in stock price for the test

pnl_explain_test('call', S, K, T, r, sigma, price_change)

Initial Price: 10.45
New Price: 11.10
Greek-based PnL: 0.65
Revaluation-based PnL: 0.65


## Digital Option test
This test check the consistency of the digital option pricer.

In [7]:

def digital_option_price(option_type, S, K, T, r, sigma):
    """
    Calculate the price of a digital option using the Black-Scholes model.

    Parameters:
    option_type (str): 'call' for digital call option, 'put' for digital put option
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)

    Returns:
    float: price of the digital option
    """
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    if option_type == 'call':
        price = np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return price

def digital_options_test(S, K, T, r, sigma):
    """
    Test the consistency of the digital option pricer.

    Parameters:
    S (float): current stock price
    K (float): option strike price
    T (float): time to expiration in years
    r (float): risk-free interest rate (annualized)
    sigma (float): volatility of the underlying stock (annualized)
    """
    digital_call_price = digital_option_price('call', S, K, T, r, sigma)
    digital_put_price = digital_option_price('put', S, K, T, r, sigma)

    print(f"Digital Call Option Price: {digital_call_price:.4f}")
    print(f"Digital Put Option Price: {digital_put_price:.4f}")

    # Check consistency
    if digital_call_price + digital_put_price > 1:
        print("Inconsistency detected: The sum of digital call and put prices exceeds 1.")
    else:
        print("Consistency check passed: The sum of digital call and put prices is valid.")

# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (1 year)
r = 0.05 # Risk-free interest rate (5%)
sigma = 0.2 # Volatility (20%)

digital_options_test(S, K, T, r, sigma)

Digital Call Option Price: 0.5323
Digital Put Option Price: 0.4189
Consistency check passed: The sum of digital call and put prices is valid.
